In [50]:
import pandas as pd
import numpy as np
import requests

url = "https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations=ALP%2CBLC%2CBLK%2CBLS%2CBSK%2CCAP%2CCSL%2CDAN%2CDDM%2CEBB%2CEP5%2CFDC%2CFLL%2CGIN%2CGKS%2CGNL%2CHGM%2CHOR%2CHRS%2CHVN%2CHYS%2CLBD%2CLVM%2CLVT%2CMDW%2CMNT%2CMRL%2CMSK%2CPSN%2CRBB%2CRBP%2CRP2%2CSDW%2CSIL%2CSLI%2CSPS%2CSPT%2CSTR%2CTCC%2CTNY%2CTUM%2CVRG%2CWC3%2CWHW&SensorNums=18&dur_code=D&Start=2013-01-01&End=2023-04-14"
response = requests.get(url).json()


In [33]:
#FULL RAW DF
station_list = []
date_list = []
value_list = []

for station in response:
    if "obsDate" in station:
        station_id = station['stationId']
        date = pd.to_datetime(station['obsDate'])
        value = station['value']
        station_list.append(station_id)
        date_list.append(date)
        value_list.append(value)

snow_df = pd.DataFrame({
    "Date" : date_list,
    "Station ID" : station_list,
    "Snow Depth (in)" : value_list,
})

snow_df["Month"] = snow_df["Date"].dt.month
snow_df["Year"] = snow_df["Date"].dt.year

In [34]:
snow_df["Snow Depth (in)"] = snow_df["Snow Depth (in)"].clip(lower = 0)
0

0

In [35]:
#SET SEASONS
winter_bin = [0,4,10,12]
labels = ["Winter_End", "Summer", "Winter_Start"]

snow_binned = snow_df.copy()
snow_binned["Season"] = pd.cut(x = snow_binned["Month"], bins =  winter_bin, labels = labels)

snow_season = snow_binned.sort_values(["Year", "Month"])

season_list = []
working_year = 2013

for row in snow_season.iterrows():
    row_year = row[1][4]
    row_season = row[1][5]

    if (row_season == "Winter_End") & (row_year == working_year):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Winter_Start") & (row_year == (working_year - 1)):
        season_list.append(f"Winter {working_year - 1} {working_year}")
    elif (row_season == "Summer"):
        season_list.append(f"Summer {working_year - 1} {working_year}")
    else:
        season_list.append(f"na")

    if (row_year == (working_year + 1)):
        working_year = working_year + 1

snow_season["Season"] = season_list

In [121]:
winter_only_mask = snow_season["Season"].str.contains("Winter")
snow_season_winter_only = snow_season.loc[winter_only_mask]
snow_season_grouped = snow_season_winter_only.groupby(["Season"])
grouplist = ["Date", "Month", "Year", "Station ID", "Snow Depth (in)"]
snow_season_grouped = snow_season_grouped[grouplist]
means = snow_season_grouped.mean()
maxes = snow_season_grouped.max()
years = snow_season_grouped.max()
snow_season_grouped.max().head(10)

,Date,Month,Year,Station ID,Snow Depth (in)
Season,,,,,
Winter 2012 2013,2013-04-30,4,2013,WHW,244
Winter 2013 2014,2014-04-30,4,2014,WHW,295
Winter 2014 2015,2015-04-30,4,2015,WHW,313
Winter 2015 2016,2016-04-30,4,2016,WHW,311
Winter 2016 2017,2017-04-30,4,2017,WHW,312
Winter 2017 2018,2018-04-30,4,2018,WHW,202
Winter 2018 2019,2019-04-30,4,2019,WHW,2317
Winter 2019 2020,2020-04-30,4,2020,WC3,332
Winter 2020 2021,2021-04-30,4,2021,WHW,322


In [123]:
seasons = snow_season_winter_only["Season"].unique()

In [117]:
mean_list = means["Snow Depth (in)"].tolist()
max_list = maxes["Snow Depth (in)"].tolist()
years_list = years["Year"].tolist()
print(mean_list)
print(max_list)
print(years_list)

[50.96466782781786, 38.419697685403875, 23.424023874658044, 55.132296650717706, 94.23593287265548, 28.860123281175913, 88.90349376985097, 40.14274651522974, 43.84542437174016, 42.99229101421344, 118.71714044469462]
[244, 295, 313, 311, 312, 202, 2317, 332, 322, 329, 334]
[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [124]:
final_snow_df = pd.DataFrame ({
    "Year": years_list,
    "Season" : seasons,
    "Mean Snow Depth (in)" : mean_list,
    "Max Snow Depth (in)" : max_list
})

final_snow_df

,Year,Season,Mean Snow Depth (in),Max Snow Depth (in)
0,2013,Winter 2012 2013,50.964668,244
1,2014,Winter 2013 2014,38.419698,295
2,2015,Winter 2014 2015,23.424024,313
3,2016,Winter 2015 2016,55.132297,311
4,2017,Winter 2016 2017,94.235933,312
5,2018,Winter 2017 2018,28.860123,202
6,2019,Winter 2018 2019,88.903494,2317
7,2020,Winter 2019 2020,40.142747,332
8,2021,Winter 2020 2021,43.845424,322
9,2022,Winter 2021 2022,42.992291,329
